In [21]:
import pandas as pd
import ast
import phonenumbers
import ipywidgets as widgets
from IPython.display import display
import os

In [22]:
# --- Helper Functions ---
def parse_utm(x):
    if pd.isna(x):
        return None
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return None
    return None

def smart_parse(num):
    s = str(num).strip()
    if not s:
        return None
    if not s.startswith("+"):
        s = "+" + s
    try:
        parsed = phonenumbers.parse(s, None)
        if phonenumbers.is_valid_number(parsed):
            return phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164).replace("+", "")
    except:
        return None
    return None


In [23]:
# --- File Upload Widgets ---
crm_widget = widgets.FileUpload(accept='.xlsx', multiple=False)
dialer_widget = widgets.FileUpload(accept='.xlsx', multiple=False)
print("Upload CRM file:")
display(crm_widget)
print("Upload Dialer file:")
display(dialer_widget)

import io

def get_df_from_widget(widget):
    if len(widget.value) == 0:
        return None
    file_info = list(widget.value.values())[0]
    content = file_info['content']
    return pd.read_excel(io.BytesIO(content))

Upload CRM file:


FileUpload(value=(), accept='.xlsx', description='Upload')

Upload Dialer file:


FileUpload(value=(), accept='.xlsx', description='Upload')

In [25]:
crm_widget = pd.read_excel(r"C:\Users\Akhil\Downloads\user_exports_01_11_2025 (1).xlsx")
dialer_widget = pd.read_excel(r"C:\Users\Akhil\Downloads\Call_log_20251103_0.xlsx")

c:\Users\Akhil\Office\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
df_crm = crm_widget
df_dialer = dialer_widget

if df_crm is None or df_dialer is None:
    print("Please upload both files.")
else:
    df_crm.columns = df_crm.columns.str.lower()
    if "utm_hit" in df_crm.columns:
        df_crm["utm_hit"] = df_crm["utm_hit"].apply(parse_utm)
        utm_df = pd.json_normalize(df_crm["utm_hit"]).add_prefix("utm_hit_")
        df_con = pd.concat([df_crm.drop(columns=["utm_hit"]), utm_df], axis=1)
    else:
        df_con = df_crm.copy()

    df_con["cleaned_phone"] = df_con["phone"].apply(smart_parse)


In [29]:
df_dialer

,cleaned_phone,account,start time,queue duration,end time,call status,answer_duration_sec,queue_sec,total_duration_sec,answer_duration_hms,total_duration_hms
0,917696370192,NaN,2025-11-03 13:01:07.874,2025-11-03 00:00:00,2025-11-03 13:01:07.884,Stopped at IVR,0.010,0,0.010,0,0
1,None,jeevan,2025-11-03 13:01:06.906,2025-11-03 00:00:18,2025-11-03 13:01:25.802,Missed,18.896,18,36.896,0,0
2,None,Shreya,2025-11-03 13:00:53.316,2025-11-03 00:00:08,2025-11-03 13:01:32.871,Answered,39.555,8,47.555,0,0
3,None,Shreya,2025-11-03 13:00:11.487,2025-11-03 00:00:02,2025-11-03 13:00:14.371,Missed,2.884,2,4.884,0,0
4,None,Shreya,2025-11-03 12:59:34.571,2025-11-03 00:00:07,2025-11-03 12:59:52.330,Answered,17.759,7,24.759,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8769,919205803269,jeevan,2025-10-06 10:52:54.055,2025-11-03 00:00:00,2025-10-06 10:52:54.690,Missed,0.635,0,0.635,0,0
8770,919819817552,jeevan,2025-10-06 10:50:57.781,2025-11-03 00:00:02,2025-10-06 10:51:00.224,Missed,2.443,2,4.443,0,0
8771,919819817552,jeevan,2025-10-06 10:50:47.263,2025-11-03 00:00:08,2025-10-06 10:50:55.414,Missed,8.151,8,16.151,0,0
8772,919916946165,NaN,2025-10-06 09:46:45.243,2025-11-03 00:00:00,2025-10-06 09:46:45.254,Stopped at IVR,0.011,0,0.011,0,0


In [28]:
df_dialer.columns = df_dialer.columns.str.lower()
df_dialer = df_dialer[['customer number','account','start time','queue duration','end time','call status']]
df_dialer["start time"] = pd.to_datetime(df_dialer["start time"])
df_dialer["end time"] = pd.to_datetime(df_dialer["end time"])
df_dialer["queue duration"] = pd.to_datetime(df_dialer["queue duration"])
df_dialer["answer_duration_sec"] = (df_dialer["end time"] - df_dialer["start time"]).dt.total_seconds()
df_dialer["queue_sec"] = (
    df_dialer["queue duration"].dt.hour * 3600 +
    df_dialer["queue duration"].dt.minute * 60 +
    df_dialer["queue duration"].dt.second
)
df_dialer["total_duration_sec"] = df_dialer["answer_duration_sec"] + df_dialer["queue_sec"]
df_dialer["answer_duration_hms"] = pd.to_timedelta(df_dialer["answer_duration_sec"], unit="s").astype(str).str.split().str[0]
df_dialer["total_duration_hms"] = pd.to_timedelta(df_dialer["total_duration_sec"], unit="s").astype(str).str.split().str[0]
df_dialer = df_dialer.rename(columns={'customer number':'cleaned_phone'})
df_dialer["cleaned_phone"] = df_dialer["cleaned_phone"].apply(smart_parse)


C:\Users\Akhil\AppData\Local\Temp\ipykernel_31040\1268606280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dialer["start time"] = pd.to_datetime(df_dialer["start time"])
C:\Users\Akhil\AppData\Local\Temp\ipykernel_31040\1268606280.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dialer["end time"] = pd.to_datetime(df_dialer["end time"])
C:\Users\Akhil\AppData\Local\Temp\ipykernel_31040\1268606280.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling b

In [30]:
df_calls = df_con.merge(df_dialer, on="cleaned_phone", how="left")
df_calls["first_name"] = df_calls["full_name"].str.split().str[0] if "full_name" in df_calls.columns else ""
df_calls["duration_sec"] = (df_calls["end time"] - df_calls["start time"]).dt.total_seconds()
df_calls.loc[df_calls["call status"] == "Missed", "duration_sec"] = 0

crm_unique_phones = set(df_con["cleaned_phone"].dropna().unique())
dialled_unique_phones = set(df_dialer["cleaned_phone"].dropna().unique())
untouched_phones = crm_unique_phones - dialled_unique_phones
contacted_phones = dialled_unique_phones

df_con["contacted"] = df_con["cleaned_phone"].isin(contacted_phones)
df_con["untouched"] = df_con["cleaned_phone"].isin(untouched_phones)


In [31]:
if "utm_hit_utmSource" in df_con.columns and "utm_hit_utmCampaign" in df_con.columns:
    total_leads = (
        df_con.groupby(["utm_hit_utmSource", "utm_hit_utmCampaign"])["cleaned_phone"]
        .nunique()
        .reset_index(name="total_leads")
    )

    def classify_status(gr):
        if 'Answered' in gr.values:
            return 'Answered'
        elif 'Missed' in gr.values:
            return 'Missed'
        else:
            return 'None'

    lead_status = (
        df_calls.groupby(["utm_hit_utmSource", "utm_hit_utmCampaign", "cleaned_phone"])["call status"]
        .agg(classify_status)
        .reset_index()
    )

    summary = (
        lead_status
        .groupby(["utm_hit_utmSource", "utm_hit_utmCampaign", "call status"])["cleaned_phone"]
        .nunique()
        .unstack(fill_value=0)
        .reset_index()
    )

    summary = summary.rename(columns={'Answered': 'answered_leads', 'Missed': 'missed_leads', 'None': 'other_leads'})
    summary["dialled_leads"] = summary.get('answered_leads', 0) + summary.get('missed_leads', 0)

    campaign_engagement = total_leads.merge(summary, on=["utm_hit_utmSource", "utm_hit_utmCampaign"], how="left").fillna(0)
    campaign_engagement["untouched_leads"] = campaign_engagement["total_leads"] - campaign_engagement["dialled_leads"]
    campaign_engagement["contact_rate_%"] = (
        (campaign_engagement["dialled_leads"] / campaign_engagement["total_leads"]) * 100
    ).round(1)
    campaign_engagement["answer_rate_%"] = campaign_engagement.apply(
        lambda x: (x["answered_leads"] / x["dialled_leads"] * 100) if x["dialled_leads"] > 0 else 0, axis=1
    ).round(1)

    print("Campaign Engagement Summary:")
    display(campaign_engagement)
else:
    print("UTM columns not found.")


Campaign Engagement Summary:


,utm_hit_utmSource,utm_hit_utmCampaign,total_leads,other_leads,dialled_leads,untouched_leads,contact_rate_%,answer_rate_%
0,XNAMETA,120232736965860173,0,0.0,0.0,0.0,NaN,0
1,XNAMETA,120232737224730173,0,0.0,0.0,0.0,NaN,0
2,an,TFX%20%7C%20Trading%20%7C%20Spaxads,2,2.0,0.0,2.0,0.0,0
3,fb,TFX%20%7C%20Trading%20%7C%20Spaxads,0,0.0,0.0,0.0,NaN,0
4,growth,growthan,3,3.0,0.0,3.0,0.0,0
5,growth,gttechtraderji,0,0.0,0.0,0.0,NaN,0
6,ig,TFX%20%7C%20Trading%20%7C%20Spaxads,0,0.0,0.0,0.0,NaN,0
7,meta-lf,120236058078790282,0,0.0,0.0,0.0,NaN,0
8,meta-lf,120236237590140282,0,0.0,0.0,0.0,NaN,0
9,meta-lk,120236057552270282,0,0.0,0.0,0.0,NaN,0


In [33]:
dialer_summary = (
    df_calls.groupby(["cleaned_phone", "first_name", "account"])
    .agg(
        answered_calls=("call status", lambda x: (x == "Answered").sum()),
        missed_calls=("call status", lambda x: (x == "Missed").sum()),
        total_duration_sec=("duration_sec", "sum"),
        answered_duration_sec=("duration_sec", lambda x: x[df_calls.loc[x.index, "call status"] == "Answered"].sum())
    ).reset_index()
)

dialer_summary["answered_duration_hms"] = pd.to_timedelta(dialer_summary["answered_duration_sec"], unit="s").astype(str).str.split().str[-1]
dialer_summary["total_duration_hms"] = pd.to_timedelta(dialer_summary["total_duration_sec"], unit="s").astype(str).str.split().str[-1]

dialer_summary = dialer_summary[["cleaned_phone", "first_name", "account", "answered_calls", "missed_calls", "answered_duration_hms", "total_duration_hms"]]
print("Dialer Summary:")
display(dialer_summary)


Dialer Summary:


,cleaned_phone,first_name,account,answered_calls,missed_calls,answered_duration_hms,total_duration_hms


In [34]:
phone_choices = dialer_summary["cleaned_phone"].dropna().unique().tolist()
if phone_choices:
    phone_dropdown = widgets.Dropdown(options=phone_choices, description='Phone:')
    display(phone_dropdown)

    def display_call_logs(change):
        selected_phone = change['new']
        call_details = df_calls[df_calls["cleaned_phone"] == selected_phone][["start time", "end time", "call status", "answer_duration_hms", "total_duration_hms"]]
        print(f"Detailed Calls for {selected_phone}:")
        display(call_details)

    phone_dropdown.observe(display_call_logs, names='value')


In [35]:
source_connectivity = (
    df_calls.groupby(["utm_hit_utmSource", "call status"])
    .size().unstack(fill_value=0).reset_index()
    .rename(columns={"Answered": "answered_calls", "Missed": "missed_calls"})
)
source_connectivity["total_calls"] = source_connectivity["answered_calls"] + source_connectivity["missed_calls"]
source_connectivity["connectivity_rate"] = (source_connectivity["answered_calls"] / source_connectivity["total_calls"]).round(2)

print("Connectivity Rate by Source:")
display(source_connectivity)


Connectivity Rate by Source:


call status,utm_hit_utmSource,answered_calls,missed_calls,Stopped at IVR,total_calls,connectivity_rate
0,IG1015,9600,18912,72,28512,0.34
1,IG1031,1200,2364,9,3564,0.34
2,IG1032,400,788,3,1188,0.34
3,IG1035,400,788,3,1188,0.34
4,IG1040,400,788,3,1188,0.34
5,MSWEsco,24400,48068,183,72468,0.34
6,TFWESocioo,2400,4728,18,7128,0.34
7,TG01,2400,4728,18,7128,0.34
8,TG02,15200,29944,114,45144,0.34
9,TG03,2000,3940,15,5940,0.34
